In [1]:
import os
import dotenv
dotenv.load_dotenv()

True

In [2]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_google_genai import GoogleGenerativeAI
from datetime import date
from langchain.agents import tool
import langchain
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

c:\Users\sharu\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# from langchain.llms.bedrock import Bedrock
# import boto3

In [4]:
langchain.debug=True

In [5]:
generation_config = {
  "temperature": 0,
  "top_p": 0.95,
  "top_k": 64,
  "response_mime_type": "text/plain",
}

In [6]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=os.environ['GOOGLE_API_KEY'],generation_config=generation_config)

In [7]:
# client = boto3.client('bedrock-runtime', region_name='us-east-1')

# llm=Bedrock(client=client,
#             model_id="anthropic.claude-instant-v1",
#             model_kwargs={
#                 "max_tokens_to_sample":500,
#                 "temperature":0,
#                 "top_p":0.9
#             }
#         )

In [9]:
vectordb=None

In [10]:
from flask import Flask, render_template_string
import threading
app = Flask(__name__)
thread=None

@tool
def host_webpage(html_content):
    """
    Returns the link where question webpage hosted, use this after \
    questions are generated in html form format. \
    The input should always be an html form code, \
    and this function will always return link to webpage hosted  \
    """

    @app.route('/')
    def display_html():
        return render_template_string(html_content)

    def run_app():
        app.run(host='localhost', port=5000, debug=False)

    # Start Flask in a separate thread
    global thread
    thread = threading.Thread(target=run_app)
    thread.start()

    # Return the link to the page
    return "http://localhost:5000/"

In [11]:
@tool
def get_revelent_context(topic: str) -> str:
    """Returns the relevent context to generate questions, use this after \
    the user have choosed the topic to generate questions. \
    The input should always be an topic that user mentioned, \
    and this function will always return context string  \
    """
    
    relevent_doc=vectordb.similarity_search(topic,k=5)
    context="\n\n".join([doc.page_content for doc in relevent_doc])

    return str(context)

In [12]:
@tool
def get_topics_from_uploaded_document(text: str) -> str:
    """Returns the topics in the uploaded documents, use this after \
    document loaded successfully. \
    The input should always be an empty string, \
    and this function will always return string of topics \
    ."""
    # return str(date.today())
    return str("aws workflows, aws redshift copy")

In [13]:
@tool
def upload_doc(text: str) -> str:
    """loads the document to memory. Use only when user mentioned i want to upload something\
       or else don't use.The input should always be an empty string. \
       and this function always returns status of the upload."""
    # return str(date.today())

    path=input("please enter path")
    print("Loading the data.....")
    loader = PyPDFLoader(path)
    data = loader.load()
    docs=text_splitter.split_documents(data)
    global vectordb 
    vectordb= Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        # persist_directory='docs/chroma/'
    )
    return str("uploaded successfully")

In [14]:
tools=[upload_doc,get_topics_from_uploaded_document,get_revelent_context,host_webpage]

In [15]:
dir(AgentType)

['CHAT_CONVERSATIONAL_REACT_DESCRIPTION',
 'CHAT_ZERO_SHOT_REACT_DESCRIPTION',
 'CONVERSATIONAL_REACT_DESCRIPTION',
 'OPENAI_FUNCTIONS',
 'OPENAI_MULTI_FUNCTIONS',
 'REACT_DOCSTORE',
 'SELF_ASK_WITH_SEARCH',
 'STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION',
 'ZERO_SHOT_REACT_DESCRIPTION',
 '__class__',
 '__doc__',
 '__members__',
 '__module__']

In [16]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    conversational=True,
    max_iterations=5,
    verbose = True)

c:\Users\sharu\anaconda3\envs\myenv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [17]:
template=""" 
Assistant is a large language model trained.
Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

You are an user-friendly MCQ Generating Assistant. Answer the human message accordingly and generate multiple-choice questions (MCQs) as best you can.
always follow these steps:

1. Greet the user and ask for the source of the content for the questions (Upload a PDF document, Provide a URL to a web page, Enter text directly).
2. Based on the chosen source, take the appropriate action:
    - Upload: Call `upload_doc` with the user-uploaded document.
    - URL: Access the web page and extract content.
    - Text: Use the provided text directly.
3. Ask the user for the difficulty level of the questions (e.g., Easy, Medium, Hard).
4. Ask for the Topic and generate 5 Question on the selected topic.
5. Generate questions in a HTML form format.
6. Call `host_webpage` with the generated HTML code and provide the link to the user.

"""

In [19]:
agent.agent.llm_chain.prompt.template=template

In [20]:
print(agent.agent.llm_chain.prompt.template)

 
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of task

In [57]:
chat_history=[]
response=agent({"input": "hi", "chat_history": chat_history})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "hi",
  "chat_history": []
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "hi",
  "chat_history": [],
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:GoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are const

In [58]:
chat_history.extend(
    [
        HumanMessage(content=response['input']),
        AIMessage(content=response["output"]),
    ]
)

In [59]:
chat_history

[HumanMessage(content='hi'),
 AIMessage(content="Hi there! 👋  I'm ready to help you create some Multiple Choice Questions (MCQs).  To get started, tell me what you want to use as the source for the questions.  Do you want to:\n\n1. **Upload a PDF document?** \n2. **Provide a URL to a web page?** \n3. **Enter text directly?**")]

In [60]:
response=agent({"input": "i want to upload a document", "chat_history": chat_history})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:GoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a 

In [61]:
chat_history.extend(
    [
        HumanMessage(content=response['input']),
        AIMessage(content=response["output"]),
    ]
)


In [62]:
chat_history

[HumanMessage(content='hi'),
 AIMessage(content="Hi there! 👋  I'm ready to help you create some Multiple Choice Questions (MCQs).  To get started, tell me what you want to use as the source for the questions.  Do you want to:\n\n1. **Upload a PDF document?** \n2. **Provide a URL to a web page?** \n3. **Enter text directly?**"),
 HumanMessage(content='i want to upload a document'),
 AIMessage(content="Great! I've successfully uploaded your document. Now, let's figure out what you want to focus on. Based on your document, I've identified the following topics:  \\n\\n1. **aws workflows** \\n2. **aws redshift copy** \\n\\nWhich topic would you like to create MCQs for?")]

In [63]:
response=agent({"input": "i would like to choose aws redshift copy to create multiple choice questions", "chat_history": chat_history})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:GoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a 

In [64]:
chat_history.extend(
    [
        HumanMessage(content=response['input']),
        AIMessage(content=response["output"]),
    ]
)

In [65]:
chat_history

[HumanMessage(content='hi'),
 AIMessage(content="Hi there! 👋  I'm ready to help you create some Multiple Choice Questions (MCQs).  To get started, tell me what you want to use as the source for the questions.  Do you want to:\n\n1. **Upload a PDF document?** \n2. **Provide a URL to a web page?** \n3. **Enter text directly?**"),
 HumanMessage(content='i want to upload a document'),
 AIMessage(content="Great! I've successfully uploaded your document. Now, let's figure out what you want to focus on. Based on your document, I've identified the following topics:  \\n\\n1. **aws workflows** \\n2. **aws redshift copy** \\n\\nWhich topic would you like to create MCQs for?"),
 HumanMessage(content='i would like to choose aws redshift copy to create multiple choice questions'),
 AIMessage(content='Awesome! I\'ve got the information on "aws redshift copy". Now, tell me, what difficulty level would you like the MCQs to be? (Easy, Medium, Hard)')]

In [66]:
response=agent({"input": "generate easy questions", "chat_history": chat_history})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:GoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a 

In [67]:
chat_history.extend(
    [
        HumanMessage(content=response['input']),
        AIMessage(content=response["output"]),
    ]
)

In [68]:
chat_history

[HumanMessage(content='hi'),
 AIMessage(content="Hi there! 👋  I'm ready to help you create some Multiple Choice Questions (MCQs).  To get started, tell me what you want to use as the source for the questions.  Do you want to:\n\n1. **Upload a PDF document?** \n2. **Provide a URL to a web page?** \n3. **Enter text directly?**"),
 HumanMessage(content='i want to upload a document'),
 AIMessage(content="Great! I've successfully uploaded your document. Now, let's figure out what you want to focus on. Based on your document, I've identified the following topics:  \\n\\n1. **aws workflows** \\n2. **aws redshift copy** \\n\\nWhich topic would you like to create MCQs for?"),
 HumanMessage(content='i would like to choose aws redshift copy to create multiple choice questions'),
 AIMessage(content='Awesome! I\'ve got the information on "aws redshift copy". Now, tell me, what difficulty level would you like the MCQs to be? (Easy, Medium, Hard)'),
 HumanMessage(content='generate easy questions'),
 

In [69]:
response=agent({"input": "nothing from myside just generate MCQ on What you have now", "chat_history": chat_history})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:GoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a 

In [73]:
chat_history.extend(
    [
        HumanMessage(content=response['input']),
        AIMessage(content=response["output"]),
    ]
)

In [74]:
chat_history

[HumanMessage(content='hi'),
 AIMessage(content="Hi there! 👋  I'm ready to help you create some Multiple Choice Questions (MCQs).  To get started, tell me what you want to use as the source for the questions.  Do you want to:\n\n1. **Upload a PDF document?** \n2. **Provide a URL to a web page?** \n3. **Enter text directly?**"),
 HumanMessage(content='i want to upload a document'),
 AIMessage(content="Great! I've successfully uploaded your document. Now, let's figure out what you want to focus on. Based on your document, I've identified the following topics:  \\n\\n1. **aws workflows** \\n2. **aws redshift copy** \\n\\nWhich topic would you like to create MCQs for?"),
 HumanMessage(content='i would like to choose aws redshift copy to create multiple choice questions'),
 AIMessage(content='Awesome! I\'ve got the information on "aws redshift copy". Now, tell me, what difficulty level would you like the MCQs to be? (Easy, Medium, Hard)'),
 HumanMessage(content='generate easy questions'),
 

In [77]:
response=agent({"input": "okay now host in a webpage", "chat_history": chat_history})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:GoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a 

 * Running on http://localhost:5000
Press CTRL+C to quit


[llm/end] [chain:AgentExecutor > chain:LLMChain > llm:GoogleGenerativeAI] [1.57s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Thought: Do I need to use a tool? No\nAI: Here's the link to your webpage with the MCQs: http://localhost:5000/ \n",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [chain:AgentExecutor > chain:LLMChain] [1.58s] Exiting Chain run with output:
{
  "text": "Thought: Do I need to use a tool? No\nAI: Here's the link to your webpage with the MCQs: http://localhost:5000/ \n"
}
[chain/end] [chain:AgentExecutor] [8.77s] Exiting Chain run with output:
{
  "output": "Here's the link to your webpage with the MCQs: http://localhost:5000/"
}


In [37]:
chat_history.extend(
    [
        HumanMessage(content=response['input']),
        AIMessage(content=response["output"]),
    ]
)

In [38]:
chat_history

[HumanMessage(content='hi'),
 AIMessage(content="Hi there! 👋  I can help you create multiple-choice questions.  To get started, please tell me how you'd like to provide me with the content for the questions.  Would you like to:\n\n1. **Upload a PDF document**\n2. **Provide a URL to a web page**\n3. **Enter text directly**"),
 HumanMessage(content='i will like to upload a  document'),
 AIMessage(content="Great! Now that your document is uploaded, let's move on.  What are the topics covered in the document?  I'll need this information to create your MCQs."),
 HumanMessage(content='ok what are topics there'),
 AIMessage(content="Great!  We've got two topics:  **aws workflows** and **aws redshift copy**.  Now, what difficulty level would you like for the questions?  (Easy, Medium, Hard)"),
 HumanMessage(content='the difficulti level is medium'),
 AIMessage(content="Okay, we'll go with medium difficulty.  Now, which of the two topics would you like to focus on: **aws workflows** or **aws re

In [41]:
response=agent({"input": "okay now using this information generate multiple choice questions", "chat_history": chat_history})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:GoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "System:  \nAssistant is a large language model trained.\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and pr

In [42]:
chat_history.extend(
    [
        HumanMessage(content=response['input']),
        AIMessage(content=response["output"]),
    ]
)

In [43]:
chat_history

[HumanMessage(content='hi'),
 AIMessage(content="Hi there! 👋  I can help you create multiple-choice questions.  To get started, please tell me how you'd like to provide me with the content for the questions.  Would you like to:\n\n1. **Upload a PDF document**\n2. **Provide a URL to a web page**\n3. **Enter text directly**"),
 HumanMessage(content='i will like to upload a  document'),
 AIMessage(content="Great! Now that your document is uploaded, let's move on.  What are the topics covered in the document?  I'll need this information to create your MCQs."),
 HumanMessage(content='ok what are topics there'),
 AIMessage(content="Great!  We've got two topics:  **aws workflows** and **aws redshift copy**.  Now, what difficulty level would you like for the questions?  (Easy, Medium, Hard)"),
 HumanMessage(content='the difficulti level is medium'),
 AIMessage(content="Okay, we'll go with medium difficulty.  Now, which of the two topics would you like to focus on: **aws workflows** or **aws re

In [44]:
response=agent({"input": "can you host the same on webpage", "chat_history": chat_history})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:GoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "System:  \nAssistant is a large language model trained.\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and pr

 * Running on http://localhost:5000
Press CTRL+C to quit


[llm/end] [chain:AgentExecutor > chain:LLMChain > llm:GoogleGenerativeAI] [1.77s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{\n    \"action\": \"Final Answer\",\n    \"action_input\": \"I've hosted the multiple-choice questions you requested on a webpage. You can access it here: http://localhost:5000/\"\n}\n```",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [chain:AgentExecutor > chain:LLMChain] [1.77s] Exiting Chain run with output:
{
  "text": "```json\n{\n    \"action\": \"Final Answer\",\n    \"action_input\": \"I've hosted the multiple-choice questions you requested on a webpage. You can access it here: http://localhost:5000/\"\n}\n```"
}
[chain/end] [chain:AgentExecutor] [8.51s] Exiting Chain run with output:
{
  "output": "I've hosted the multiple-choice questions you requested on a webpage. You can access it here: http://localhost:5000/"
}
